In [1]:
import time
import requests
import json
import os
from getpass import getpass

# Get your API key from https://cloud.ionq.com/settings/keys
api_key = os.getenv('IONQ_API_KEY') or getpass('Enter your IonQ API key: ')

In [2]:
def submit_job(headers, data):
    url = "https://api.ionq.co/v0.3/jobs"
    response = requests.post(url, headers=headers, json=data)
    response_json = response.json()
    assert response.status_code == 200, f"Error: {response_json.get('message', 'Unknown error')}"
    return response_json["id"]

def query_job(job_id, headers):
    url = f"https://api.ionq.co/v0.3/jobs/{job_id}"
    response = requests.get(url, headers=headers)
    response_json = response.json()
    assert response.status_code == 200, f"Error: {response_json.get('message', 'Unknown error')}"
    return response_json["status"]

def get_job_results(job_id, headers):
    url = f"https://api.ionq.co/v0.3/jobs/{job_id}/results"
    response = requests.get(url, headers=headers)
    response_json = response.json()
    assert response.status_code == 200, f"Error: {response_json.get('message', 'Unknown error')}"
    assert response_json == {"0": 0.500000000, "3": 0.500000000}, f"Unexpected results: {response_json}"
    return response_json

In [3]:
with open("data.json", "r") as file:
    data = json.load(file)

headers = {
    "Authorization": f"apiKey {api_key}",
    "Content-Type": "application/json",
}

job_id = submit_job(headers, data)

status = "ready"
while status != "completed":
    time.sleep(1)  # wait for 1 second before querying again
    status = query_job(job_id, headers)

results = get_job_results(job_id, headers)
print(results)

{'0': 0.5, '3': 0.5}
